# Master Thesis
## Factor models docs
### v1.0 - gathering resources for factor models

## Sorted:
Available packages in R to take inspiration from:
 - [YieldCurve](https://cran.r-project.org/web/packages/YieldCurve/YieldCurve.pdf)
    - [Nelson.Siegel](https://www.rdocumentation.org/packages/YieldCurve/versions/4.1/topics/Nelson.Siegel) function, its code [here](https://rdrr.io/rforge/YieldCurve/src/R/Nelson.Siegel.R)

## Sort out:

## Factor models in R

* [DEnss.pdf](https://cran.r-project.org/web/packages/NMOF/vignettes/DEnss.pdf)
* [download.cgi](https://editorialexpress.com/cgi-bin/conference/download.cgi?db_name=XVIEBFIN&paper_id=33)
* [Forecasting the yield curve with the arbitrage-free dynamic Nelson–Siegel model: Brazilian evidence - ScienceDirect](https://www.sciencedirect.com/science/article/pii/S1517758016300789)
* [Calibrating the Dynamic Nelson-Siegel Model: A Practitioner Approach - Munich Personal RePEc Archive](https://mpra.ub.uni-muenchen.de/68439/)
* [ijfs-06-00068.pdf](file:///E:/Google_Drive/Diploma_Thesis/Sources/Random_Google/Factor%20models/ijfs-06-00068.pdf)
* [The Nelson-Siegel Model: Present Application and Alternative Lambda Determination](file:///E:/Google_Drive/Diploma_Thesis/Sources/Random_Google/Factor%20models/BPTX_2013_2_11230_0_387924_0_151485.pdf)
